# Practice notebook for confidence intervals using NHANES data

This notebook will give you the opportunity to practice working with confidence intervals using the NHANES data.

You can enter your code into the cells that say "enter your code here", and you can type responses to the questions into the cells that say "Type Markdown and Latex".

Note that most of the code that you will need to write below is very similar to code that appears in the case study notebook.  You will need to edit code from that notebook in small ways to adapt it to the prompts below.

To get started, we will use the same module imports and read the data in the same way as we did in the case study:

In [82]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as sci
da = pd.read_csv("nhanes_2015_2016.csv")

da.head()

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
0,83732,1.0,NaN,1.0,1,1,62,3,1.0,5.0,...,124.0,64.0,94.8,184.5,27.8,43.3,43.6,35.9,101.1,2.0
1,83733,1.0,NaN,6.0,1,1,53,3,2.0,3.0,...,140.0,88.0,90.4,171.4,30.8,38.0,40.0,33.2,107.9,NaN
2,83734,1.0,NaN,NaN,1,1,78,3,1.0,3.0,...,132.0,44.0,83.4,170.1,28.8,35.6,37.0,31.0,116.5,2.0
3,83735,2.0,1.0,1.0,2,2,56,3,1.0,5.0,...,134.0,68.0,109.8,160.9,42.4,38.5,37.7,38.3,110.1,2.0
4,83736,2.0,1.0,1.0,2,2,42,4,1.0,4.0,...,114.0,54.0,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0


## Question 1

Restrict the sample to women between 35 and 50, then use the marital status variable [DMDMARTL](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#DMDMARTL) to partition this sample into two groups - women who are currently married, and women who are not currently married.  Within each of these groups, calculate the proportion of women who have completed college.  Calculate 95% confidence intervals for each of these proportions.

In [90]:
# enter your code here
da["RIAGENDRx"]=da.RIAGENDR.replace({1:"Male",2:"Female"})
dx=da.loc[(da.RIDAGEYR >=35)& (da.RIDAGEYR<=50) & (da.RIAGENDRx == "Female"),:]
dx["DMDMARTLx"]=da.DMDMARTL.replace({1:"Currently_Married",2:"Not_Currently_Married",3:"Not_Currently_Married",4:"Not_Currently_Married",5:"Not_Currently_Married",6:"Not_Currently_Married",77:np.nan,99:np.nan})
dx['DMDEDUC2x']=da['DMDEDUC2'].replace({5:1,1:0,2:0,3:0,4:0,7:0,9:0})
dy=dx[["DMDMARTLx","DMDEDUC2x"]].dropna()
p=pd.crosstab(dy.DMDMARTLx, dy.DMDEDUC2x)
p.columns=["Not College Graduate or above","College Graduate or above"]
p

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Not College Graduate or above,College Graduate or above
DMDMARTLx,,
Currently_Married,287,162
Not_Currently_Married,266,72



There are 449 married women, 162 of which are graduates
The proportion of married women who graduated collage is 0.36080178173719374


There are 338 unmarried women, 72 of which are graduates
The proportion of married women who didn't graduate collage is 0.21301775147928995

In [7]:
conf_int1= sm.stats.proportion_confint(162,449)
conf_int1

(0.31638193710753626, 0.4052216263668512)

In [9]:
conf_int2= sm.stats.proportion_confint(72,338)
conf_int2

(0.16936816767089768, 0.2566673352876822)

__Q1a.__ Identify which of the two confidence intervals is wider, and explain why this is the case. 

- the interval for married women is slightly wider
- I think it is because the proportion of married women who graduated is higher than non-maried women and its more closer to 0.5 as "conservative confidance interval" stats that the closer the proportion to 0.5. the wider the variance

__Q1b.__ Write 1-2 sentences summarizing these findings for an audience that does not know what a confidence interval is (the goal here is to report the substance of what you learned about how marital status and educational attainment are related, not to teach a person what a confidence interval is).

- Based on our analysis of women aged between 30 to 35 we are 95% confident that the population proportion of currently married woman who have a gollage degree is between 31.64% and 40.52% 
- in addition to women who are not currently married is between 16.94% and 25.67% 

## Question 2

Construct a 95% confidence interval for the proportion of smokers who are female. Construct a 95% confidence interval for the proportion of smokers who are male. Construct a 95% confidence interval for the **difference** between those two gender proportions.

In [57]:
da["SMQ020x"] = da.SMQ020.replace({1: 1, 2: 0, 7: np.nan, 9: np.nan}) 
f_smoker= da.loc[da.RIAGENDRx=="Female",'SMQ020x'].dropna()
f_smoker.head()

3     0.0
4     0.0
5     0.0
7     0.0
12    1.0
Name: SMQ020x, dtype: float64

In [59]:
sm.stats.proportion_confint(sum(f_smoker),len(f_smoker))


(0.2882949879861214, 0.32139545615923526)

In [66]:
m_smoker= da.loc[da.RIAGENDRx=="Male","SMQ020x"].dropna()
m_smoker.head()

0    1.0
1    1.0
2    1.0
6    1.0
8    0.0
Name: SMQ020x, dtype: float64

In [67]:
sm.stats.proportion_confint(sum(m_smoker),len(m_smoker))

(0.49458749263718593, 0.5319290347874418)

In [71]:
pm= len(m_smoker.loc[m_smoker==1])/len(m_smoker)
pm

0.5132582637123139

#### the proportion of men who smokes is 51.3%

In [73]:
pf= len(f_smoker.loc[f_smoker==1])/len(f_smoker)
pf

0.30484522207267833

#### the proportion of women who smokes is 30.48%

In [74]:
f_se = np.sqrt((pf * (1-pf))/len(f_smoker))
m_se = np.sqrt((pm * (1-pm))/len(m_smoker))
se_diff = np.sqrt(f_se**2 + m_se**2)
se_diff

0.012729881381407434

In [77]:
d = pm - pf
lcb = d - 1.96 * se_diff
ucb = d + 1.96 * se_diff
(lcb, ucb)


(0.18346247413207697, 0.2333636091471941)

__Q2a.__ Why might it be relevant to report the separate gender proportions **and** the difference between the gender proportions?

- Separate gender proportions provide insights of each gender's smoking habits.
- The proportion difference indicates that almost half of the men are smokers, while only 30% of women are smokers.


__Q2b.__ How does the **width** of the confidence interval for the difference between the gender proportions compare to the widths of the confidence intervals for the separate gender proportions?

- the confidence intervals for the differance between the two population is wider than in seperate gender proportions , this indicates that it has large margin of error

## Question 3

Construct a 95% interval for height ([BMXHT](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/BMX_I.htm#BMXHT)) in centimeters.  Then convert height from centimeters to inches by dividing by 2.54, and construct a 95% confidence interval for height in inches.  Finally, convert the endpoints (the lower and upper confidence limits) of the confidence interval from inches to back to centimeters   

In [83]:
def get_ci(col):
    
    mean = np.mean(col)
    std = np.std(col)
    df = len(col)-1
    t_star = sci.t.ppf(0.95,df)
    se = std / np.sqrt(len(col))
    lb = mean - t_star * se
    ub = mean + t_star *se 
    
    return (lb,ub)

In [84]:
get_ci(da.BMXHT)

(165.923896430169, 166.36177252805362)

In [85]:
get_ci(da.BMXHT/2.54)

(65.32436867329481, 65.49676083781632)

In [86]:
165.9238964301693/65.32436867329498

2.5400000000000005

__Q3a.__ Describe how the confidence interval constructed in centimeters relates to the confidence interval constructed in inches.

- **The confidende interval of the height in cm is 2.54 * the confidence interval of the height in inches.**

## Question 4

Partition the sample based on 10-year age bands, i.e. the resulting groups will consist of people with ages from 18-28, 29-38, etc. Construct 95% confidence intervals for the difference between the mean BMI for females and for males within each age band.

In [97]:
df = da[da['RIAGENDRx'].isin(['Female', 'Male'])].copy()

In [98]:
df["AGEGRP"]=pd.cut(df.RIDAGEYR,[18,29,39,49,59,69,80])
df.head()

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210,RIAGENDRx,AGEGRP
0,83732,1.0,NaN,1.0,1,1,62,3,1.0,5.0,...,94.8,184.5,27.8,43.3,43.6,35.9,101.1,2.0,Male,"(59, 69]"
1,83733,1.0,NaN,6.0,1,1,53,3,2.0,3.0,...,90.4,171.4,30.8,38.0,40.0,33.2,107.9,NaN,Male,"(49, 59]"
2,83734,1.0,NaN,NaN,1,1,78,3,1.0,3.0,...,83.4,170.1,28.8,35.6,37.0,31.0,116.5,2.0,Male,"(69, 80]"
3,83735,2.0,1.0,1.0,2,2,56,3,1.0,5.0,...,109.8,160.9,42.4,38.5,37.7,38.3,110.1,2.0,Female,"(49, 59]"
4,83736,2.0,1.0,1.0,2,2,42,4,1.0,4.0,...,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0,Female,"(39, 49]"


In [100]:
pr = df.groupby(["AGEGRP",'RIAGENDRx']).agg({"BMXBMI":[np.mean,np.std,np.size]}).unstack()
pr

BMXBMI                                             
                mean                  std             size       
RIAGENDRx     Female       Male    Female      Male Female   Male
AGEGRP                                                           
(18, 29]   28.082450  27.230677  7.890613  6.587966  551.0  508.0
(29, 39]   30.208211  29.772422  8.192074  6.825048  481.0  452.0
(39, 49]   30.922332  29.563409  7.911045  6.179002  511.0  402.0
(49, 59]   30.864732  29.193807  7.584018  5.974769  451.0  437.0
(59, 69]   31.029806  29.322426  7.799010  5.904651  468.0  449.0
(69, 80]   29.284897  28.214483  6.398495  5.201107  444.0  448.0

In [101]:
# Calculate the SEM for females and for males within each age band
pr["BMXBMI","sem","Female"]=pr["BMXBMI","std","Female"]/np.sqrt(pr["BMXBMI","size","Female"])
pr["BMXBMI","sem","Male"]=pr["BMXBMI","std","Male"]/np.sqrt(pr["BMXBMI","size","Male"])
pr

BMXBMI                                                         \
                mean                  std             size              sem   
RIAGENDRx     Female       Male    Female      Male Female   Male    Female   
AGEGRP                                                                        
(18, 29]   28.082450  27.230677  7.890613  6.587966  551.0  508.0  0.336151   
(29, 39]   30.208211  29.772422  8.192074  6.825048  481.0  452.0  0.373526   
(39, 49]   30.922332  29.563409  7.911045  6.179002  511.0  402.0  0.349964   
(49, 59]   30.864732  29.193807  7.584018  5.974769  451.0  437.0  0.357117   
(59, 69]   31.029806  29.322426  7.799010  5.904651  468.0  449.0  0.360509   
(69, 80]   29.284897  28.214483  6.398495  5.201107  444.0  448.0  0.303659   

                     
                     
RIAGENDRx      Male  
AGEGRP               
(18, 29]   0.292294  
(29, 39]   0.321023  
(39, 49]   0.308181  
(49, 59]   0.285812  
(59, 69]   0.278658  
(69, 80]   0.245729

In [104]:
# Calculate the mean difference of BMI between females and males within each age band, also  calculate
# its SE and the lower and upper limits of its 95% CI.
pr["BMXBMI","mean_diff",""]= pr["BMXBMI","mean","Female"] - pr["BMXBMI","mean","Male"]
pr["BMXBMI","sem_diff",""] = np.sqrt(pr["BMXBMI","sem","Female"]**2 + pr["BMXBMI","sem","Male"]**2)
pr["BMXBMI", "lcb_diff", ""] = pr["BMXBMI", "mean_diff", ""] - 1.96 * pr["BMXBMI", "sem_diff", ""] 
pr["BMXBMI", "ucb_diff", ""] = pr["BMXBMI", "mean_diff", ""] + 1.96 * pr["BMXBMI", "sem_diff", ""] 
pr["BMXBMI", "width", ""] = pr["BMXBMI", "ucb_diff", ""] - pr["BMXBMI", "lcb_diff", ""]
pr

BMXBMI                                                         \
                mean                  std             size              sem   
RIAGENDRx     Female       Male    Female      Male Female   Male    Female   
AGEGRP                                                                        
(18, 29]   28.082450  27.230677  7.890613  6.587966  551.0  508.0  0.336151   
(29, 39]   30.208211  29.772422  8.192074  6.825048  481.0  452.0  0.373526   
(39, 49]   30.922332  29.563409  7.911045  6.179002  511.0  402.0  0.349964   
(49, 59]   30.864732  29.193807  7.584018  5.974769  451.0  437.0  0.357117   
(59, 69]   31.029806  29.322426  7.799010  5.904651  468.0  449.0  0.360509   
(69, 80]   29.284897  28.214483  6.398495  5.201107  444.0  448.0  0.303659   

                                                                       
                    mean_diff  sem_diff  lcb_diff  ucb_diff     width  
RIAGENDRx      Male                                                    
AGEGRP                                                                 
(18, 29]   0.292294  0.851772  0.445459 -0.021326  1.724871  1.746198  
(29, 39]   0.321023  0.435789  0.492522 -0.529554  1.401132  1.930686  
(39, 49]   0.308181  1.358923  0.466315  0.444945  2.272902  1.827957  
(49, 59]   0.285812  1.670925  0.457407  0.774407  2.567443  1.793036  
(59, 69]   0.278658  1.707380  0.455650  0.814306  2.600454  1.786149  
(69, 80]   0.245729  1.070414  0.390630  0.304780  1.836049  1.531269

__Q4a.__ How do the widths of these confidence intervals differ?  Provide an explanation for any substantial diferences in the confidence interval widths that you see.
   - in the first two age groups from 18 to 39 years their is no statistical diferance in bmi in females and males , but the rest age groups their is an evidance that females bmi is larger than males bmi

## Question 5

Construct a 95% confidence interval for the first and second systolic blood pressure measures, and for the difference between the first and second systolic blood pressure measurements within a subject.

In [108]:
CI1= sm.stats.DescrStatsW(da.BPXSY1.dropna()).zconfint_mean()
CI1

(124.59174272058787, 125.57748520016754)

In [109]:
CI2= sm.stats.DescrStatsW(da.BPXSY2.dropna()).zconfint_mean()
CI2

(124.29493306967777, 125.27110125733216)

In [113]:
from statsmodels.stats import weightstats as sms
cm = sms.CompareMeans(sms.DescrStatsW(da.BPXSY1.dropna()),sms.DescrStatsW(da.BPXSY2.dropna()))

In [120]:
CI3= cm.tconfint_diff(usevar="unequal")
CI3

(-0.3921284617422677, 0.9953220554877552)

In [115]:
CI1[1] - CI1[0]

0.9857424795796703

In [116]:
CI2[1] - CI2[0]

0.9761681876543946

In [117]:
CI3[1] - CI3[0]

1.3874505172300229

__Q5a.__ Based on these confidence intervals, would you say that a difference of zero between the population mean values of the first and second systolic blood pressure measures is consistent with the data?

  - yes because zero is included in the interval

__Q5b.__ Discuss how the width of the confidence interval for the within-subject difference compares to the widths of the confidence intervals for the first and second measures.

 - The width of the confidence intervals for the first and second measures are very similar, however, of the confidence intervals for the within-subject difference is slightly larger by 0.4 as it has a larger margin of error

## Question 6

Construct a 95% confidence interval for the mean difference between the average age of a smoker, and the average age of a non-smoker.

In [121]:
smokers = da[da['SMQ020'] == 1]['RIDAGEYR']
non_smokers = da[~(da['SMQ020'] == 1)]['RIDAGEYR']

In [122]:
CI_S1 = sm.stats.DescrStatsW(smokers.dropna()).zconfint_mean()
CI_S1

(51.38591951147112, 52.80726720694198)

In [123]:
CI_S2 = sm.stats.DescrStatsW(non_smokers.dropna()).zconfint_mean()
CI_S2

(44.68411549118788, 45.929467646985415)

In [126]:
sms.CompareMeans(sm.stats.DescrStatsW(smokers.dropna()),sm.stats.DescrStatsW(non_smokers.dropna())).tconfint_diff(usevar="unequal")

(5.844708831852225, 7.73489474838758)